In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install streamlit pandas matplotlib seaborn pyngrok -q

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken 2x31PIaY2yj30FAtc55kKHWVhM5_3dLz4UzeatnQKYfAPoodQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile dashboard.py
# This line is for Colab. If running locally, you don't need it.
# Just save the content below as dashboard.py

import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import numpy as np

# --- Page Configuration ---
st.set_page_config(
    page_title="Advanced SVM Tuning Dashboard",
    page_icon="🚀",
    layout="wide"
)

# --- Constants and Helper Functions ---
CSV_FILE_PATH = '/content/drive/My Drive/all_svm_tuning_results.csv' # UPDATE IF YOUR CSV IS ELSEWHERE

@st.cache_data
def load_data(path):
    try:
        data = pd.read_csv(path)
        if 'Test Accuracy' in data.columns:
            data['Test Error Rate'] = 1 - data['Test Accuracy']
        return data
    except FileNotFoundError:
        st.error(f"Error: CSV file not found at '{path}'. Ensure it's in the correct location.")
        return None
    except Exception as e:
        st.error(f"Error loading data: {e}")
        return None

def parse_best_parameters(param_string):
    if pd.isna(param_string) or param_string == "": return None
    try:
        evaluated = ast.literal_eval(param_string)
        if isinstance(evaluated, list) and len(evaluated) == 2: return {'C': evaluated[0], 'gamma': evaluated[1]}
        elif isinstance(evaluated, dict):
            if 'svc__C' in evaluated and 'svc__gamma' in evaluated: return {'C': evaluated['svc__C'], 'gamma': evaluated['svc__gamma']}
            return evaluated
        return None
    except: return None

def generate_simulated_learning_curves(method_name, num_runs=30, num_iterations=10, final_mean_f1=0.67, final_std_f1=0.02):
    all_runs_history = []
    for _ in range(num_runs):
        run_history = []
        current_f1 = 0.3 + np.random.rand() * 0.1
        for i in range(num_iterations):
            improvement = (final_mean_f1 - current_f1) * (np.random.rand() * 0.3 + 0.1) * (1 - i/num_iterations)**0.5
            noise = (np.random.rand() - 0.5) * 0.02
            current_f1 += improvement + noise
            current_f1 = np.clip(current_f1, 0, 1.0)
            run_history.append(current_f1)
        run_history[-1] = np.random.normal(loc=final_mean_f1, scale=final_std_f1/2)
        run_history[-1] = np.clip(run_history[-1], 0, 1.0)
        all_runs_history.append(run_history)
    df_lc = pd.DataFrame(all_runs_history).T
    lc_summary = pd.DataFrame({
        'Iteration': range(1, num_iterations + 1),
        'Mean_Val_F1': df_lc.mean(axis=1),
        'Std_Val_F1': df_lc.std(axis=1)
    })
    lc_summary['Lower_Bound'] = lc_summary['Mean_Val_F1'] - lc_summary['Std_Val_F1']
    lc_summary['Upper_Bound'] = lc_summary['Mean_Val_F1'] + lc_summary['Std_Val_F1']
    return lc_summary

results_df = load_data(CSV_FILE_PATH)

# --- Main Application ---
st.markdown("## 🚀 Advanced SVM Hyperparameter Tuning Dashboard")

if results_df is not None:
    st.sidebar.header("Global Controls")
    all_methods = sorted(results_df['Method'].unique())
    default_compare_methods = [m for m in ['Baseline', 'GridSearch', 'PSO', 'ICA'] if m in all_methods]
    selected_methods_compare = st.sidebar.multiselect(
        "Select methods for comparison plots:",
        options=all_methods,
        default=default_compare_methods
    )

    if not selected_methods_compare:
        st.warning("Please select at least one method in the sidebar for comparison plots.")
        st.stop()
    df_to_compare = results_df[results_df['Method'].isin(selected_methods_compare)]

    st.markdown("### I. Overall Performance & Cost Comparison")
    st.markdown(f"Comparing: `{'`, `'.join(selected_methods_compare)}`")

    FIG_WIDTH = 7
    FIG_HEIGHT = 5
    TITLE_FONTSIZE = 10
    LABEL_FONTSIZE = 8

    # --- Grid of 6 plots (2 columns, 3 rows) ---
    # We will use bar plots to show mean values instead of box plots for the main metrics.
    # For Tuning Time, a box plot is still good to show distribution, but without log scale checkbox.

    row1_col1, row1_col2 = st.columns(2)
    with row1_col1:
        st.markdown("##### Average Test F1-Score")
        avg_f1_data = df_to_compare.groupby('Method')['Test F1-Score'].mean().reindex(selected_methods_compare).reset_index()
        fig_f1, ax_f1 = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
        sns.barplot(x='Method', y='Test F1-Score', data=avg_f1_data, ax=ax_f1, order=selected_methods_compare)
        ax_f1.set_title("Average Test F1-Score", fontsize=TITLE_FONTSIZE)
        ax_f1.set_xlabel("Method", fontsize=LABEL_FONTSIZE); ax_f1.set_ylabel("Average Test F1-Score", fontsize=LABEL_FONTSIZE)
        plt.xticks(rotation=45, ha='right', fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
        plt.tight_layout(); st.pyplot(fig_f1)
    with row1_col2:
        st.markdown("##### Average Test Accuracy") # This was already a bar plot, keeping it.
        avg_acc_data = df_to_compare.groupby('Method')['Test Accuracy'].mean().reindex(selected_methods_compare).reset_index()
        fig_acc, ax_acc = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
        sns.barplot(x='Method', y='Test Accuracy', data=avg_acc_data, ax=ax_acc, order=selected_methods_compare)
        ax_acc.set_title("Average Test Accuracy", fontsize=TITLE_FONTSIZE)
        ax_acc.set_xlabel("Method", fontsize=LABEL_FONTSIZE); ax_acc.set_ylabel("Average Test Accuracy", fontsize=LABEL_FONTSIZE)
        plt.xticks(rotation=45, ha='right', fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
        plt.tight_layout(); st.pyplot(fig_acc)

    row2_col1, row2_col2 = st.columns(2)
    with row2_col1:
        st.markdown("##### Average Test ROC AUC")
        avg_roc_data = df_to_compare.groupby('Method')['Test ROC AUC'].mean().reindex(selected_methods_compare).reset_index()
        fig_roc, ax_roc = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
        sns.barplot(x='Method', y='Test ROC AUC', data=avg_roc_data, ax=ax_roc, order=selected_methods_compare)
        ax_roc.set_title("Average Test ROC AUC", fontsize=TITLE_FONTSIZE)
        ax_roc.set_xlabel("Method", fontsize=LABEL_FONTSIZE); ax_roc.set_ylabel("Average Test ROC AUC", fontsize=LABEL_FONTSIZE)
        plt.xticks(rotation=45, ha='right', fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
        plt.tight_layout(); st.pyplot(fig_roc)
    with row2_col2:
        st.markdown("##### Tuning Time (s) Distribution") # Box plot is good here to show variability
        fig_time, ax_time = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
        sns.boxplot(x='Method', y='Tuning Time (s)', data=df_to_compare, ax=ax_time, order=selected_methods_compare)
        ax_time.set_yscale('log') # Defaulting to log scale for tuning time as it often varies a lot
        ax_time.set_title("Distribution of Tuning Time (s) (Log Scale)", fontsize=TITLE_FONTSIZE)
        ax_time.set_xlabel("Method", fontsize=LABEL_FONTSIZE); ax_time.set_ylabel("Tuning Time (s)", fontsize=LABEL_FONTSIZE)
        plt.xticks(rotation=45, ha='right', fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
        plt.tight_layout(); st.pyplot(fig_time)

    # The "Avg. Test Accuracy Comparison" and "Avg. Tuning Time Comparison" from previous row 2 are now redundant
    # as we've made the primary plots bar plots of means.
    # We can use this space for other useful comparisons or keep it cleaner.
    # For now, let's remove the 3rd row of this section to simplify.

    # --- Section 2: CI Algorithm Learning Curves (Simulated) ---
    st.markdown("### II. CI Algorithm Learning Curves (Simulated Data)")
    ci_methods_for_lc = [m for m in ['PSO', 'ICA'] if m in selected_methods_compare]
    if ci_methods_for_lc:
        selected_lc_method = st.selectbox("Select CI method for Learning Curve:", ci_methods_for_lc)
        actual_final_f1_stats = results_df[results_df['Method'] == selected_lc_method]['Best Training Val F1-Score']
        sim_final_mean_f1 = actual_final_f1_stats.mean() if not actual_final_f1_stats.empty else 0.67
        sim_final_std_f1 = actual_final_f1_stats.std() if not actual_final_f1_stats.empty else 0.02
        num_iterations_sim = 10
        lc_data_simulated = generate_simulated_learning_curves(selected_lc_method, num_iterations=num_iterations_sim, final_mean_f1=sim_final_mean_f1, final_std_f1=sim_final_std_f1)

        fig_lc, ax_lc = plt.subplots(figsize=(FIG_WIDTH*1.5, FIG_HEIGHT))
        ax_lc.plot(lc_data_simulated['Iteration'], lc_data_simulated['Mean_Val_F1'], marker='o', label=f'Avg. Val F1 ({selected_lc_method})')
        ax_lc.fill_between(lc_data_simulated['Iteration'], lc_data_simulated['Lower_Bound'], lc_data_simulated['Upper_Bound'], alpha=0.2, label='Std. Dev.')
        ax_lc.set_title(f"Simulated Learning Curve for {selected_lc_method}", fontsize=TITLE_FONTSIZE)
        ax_lc.set_xlabel("Iteration / Decade (Simulated)", fontsize=LABEL_FONTSIZE); ax_lc.set_ylabel("Average Validation F1-Score (Simulated)", fontsize=LABEL_FONTSIZE)
        plt.xticks(fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
        ax_lc.legend(fontsize=LABEL_FONTSIZE); ax_lc.grid(True)
        plt.tight_layout(); st.pyplot(fig_lc)
    else:
        st.info("Select PSO or ICA in sidebar to view (simulated) learning curves.")

    # --- Section 3: Error Rate Visualization ---
    st.markdown("### III. Average Test Error Rate Comparison")
    if 'Test Error Rate' in df_to_compare.columns:
        avg_err_data = df_to_compare.groupby('Method')['Test Error Rate'].mean().reindex(selected_methods_compare).reset_index()
        fig_err, ax_err = plt.subplots(figsize=(FIG_WIDTH*1.5, FIG_HEIGHT))
        sns.barplot(x='Method', y='Test Error Rate', data=avg_err_data, ax=ax_err, order=selected_methods_compare) # Changed to barplot
        ax_err.set_title("Average Test Error Rate (1 - Accuracy) by Method", fontsize=TITLE_FONTSIZE)
        ax_err.set_xlabel("Method", fontsize=LABEL_FONTSIZE); ax_err.set_ylabel("Average Test Error Rate", fontsize=LABEL_FONTSIZE)
        plt.xticks(rotation=45, ha='right', fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
        plt.tight_layout(); st.pyplot(fig_err)
    else:
        st.warning("Test Error Rate column not found. Ensure 'Test Accuracy' is in the CSV.")

    # --- Section 4: CI Algorithm Deep Dive (Parameter Distributions) ---
    st.markdown("### IV. CI Algorithm Hyperparameter Search Analysis (PSO & ICA)")
    ci_methods_params = [m for m in ['PSO', 'ICA'] if m in selected_methods_compare]
    if ci_methods_params:
        selected_ci_params_method = st.selectbox("Select CI algorithm for parameter analysis:", ci_methods_params, key="ci_param_select")
        df_ci_params_dive = results_df[results_df['Method'] == selected_ci_params_method]
        if not df_ci_params_dive.empty:
            st.markdown(f"##### Distribution of Best Hyperparameters Found by {selected_ci_params_method}")
            parsed_params = df_ci_params_dive['Best Parameters'].apply(parse_best_parameters)
            valid_params_list = [p for p in parsed_params if isinstance(p, dict) and 'C' in p and 'gamma' in p]
            if valid_params_list:
                df_params = pd.DataFrame(valid_params_list)
                param_col1, param_col2 = st.columns(2)
                with param_col1:
                    fig_c, ax_c = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT)); sns.histplot(df_params['C'], kde=True, ax=ax_c, log_scale=st.checkbox(f"Log scale for C ({selected_ci_params_method})", True, key=f"log_c_{selected_ci_params_method}_detail"))
                    ax_c.set_title(f'Distribution of Best C', fontsize=TITLE_FONTSIZE)
                    ax_c.set_xlabel("C value", fontsize=LABEL_FONTSIZE); ax_c.set_ylabel("Frequency", fontsize=LABEL_FONTSIZE)
                    plt.xticks(fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
                    plt.tight_layout(); st.pyplot(fig_c)
                with param_col2:
                    fig_g, ax_g = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT)); sns.histplot(df_params['gamma'], kde=True, ax=ax_g, log_scale=st.checkbox(f"Log scale for gamma ({selected_ci_params_method})", True, key=f"log_g_{selected_ci_params_method}_detail"))
                    ax_g.set_title(f'Distribution of Best gamma', fontsize=TITLE_FONTSIZE)
                    ax_g.set_xlabel("Gamma value", fontsize=LABEL_FONTSIZE); ax_g.set_ylabel("Frequency", fontsize=LABEL_FONTSIZE)
                    plt.xticks(fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
                    plt.tight_layout(); st.pyplot(fig_g)

                if 'Test F1-Score' in df_ci_params_dive.columns:
                    df_params_plot = df_params.copy()
                    f1_scores_for_plot = []
                    original_indices = parsed_params.dropna().index
                    for idx in original_indices:
                         if isinstance(parsed_params.loc[idx], dict) and 'C' in parsed_params.loc[idx] and 'gamma' in parsed_params.loc[idx]:
                            f1_scores_for_plot.append(df_ci_params_dive.loc[idx, 'Test F1-Score'])
                    if len(f1_scores_for_plot) == len(df_params_plot): df_params_plot['Test F1-Score'] = f1_scores_for_plot

                    fig_c_gamma, ax_c_gamma = plt.subplots(figsize=(FIG_WIDTH, FIG_HEIGHT))
                    if 'Test F1-Score' in df_params_plot.columns:
                        scatter = ax_c_gamma.scatter(df_params_plot['C'], df_params_plot['gamma'], c=df_params_plot['Test F1-Score'], cmap='viridis', alpha=0.7, s=50)
                        cbar = plt.colorbar(scatter, ax=ax_c_gamma, label='Test F1-Score')
                        cbar.ax.tick_params(labelsize=LABEL_FONTSIZE)
                        cbar.set_label('Test F1-Score', size=LABEL_FONTSIZE)
                    else: ax_c_gamma.scatter(df_params_plot['C'], df_params_plot['gamma'], alpha=0.7, s=50)
                    ax_c_gamma.set_title(f'Best (C, gamma) pairs by {selected_ci_params_method}', fontsize=TITLE_FONTSIZE)
                    ax_c_gamma.set_xlabel('C value', fontsize=LABEL_FONTSIZE); ax_c_gamma.set_ylabel('gamma value', fontsize=LABEL_FONTSIZE)
                    if st.checkbox(f"Log scales for C & gamma ({selected_ci_params_method})", True, key=f"log_cg_{selected_ci_params_method}_detail"):
                        ax_c_gamma.set_xscale('log'); ax_c_gamma.set_yscale('log')
                    plt.xticks(fontsize=LABEL_FONTSIZE); plt.yticks(fontsize=LABEL_FONTSIZE)
                    plt.tight_layout(); st.pyplot(fig_c_gamma)
            else: st.warning(f"Could not parse C/gamma for {selected_ci_params_method}.")
    else: st.info("Select PSO or ICA in sidebar to see parameter analysis.")

    st.markdown("### V. Detailed Run Data")
    with st.expander("Explore Specific Runs"):
        explore_method_detail = st.selectbox("Select method:", options=all_methods, key="explore_method_specific")
        if explore_method_detail:
            df_explore_specific = results_df[results_df['Method'] == explore_method_detail]
            if not df_explore_specific.empty:
                if explore_method_detail in ['PSO', 'ICA']:
                    available_seeds_detail = sorted(df_explore_specific['Seed'].dropna().unique().astype(int))
                    if available_seeds_detail:
                        selected_seed_detail = st.select_slider(f"Select Seed for {explore_method_detail}:", options=available_seeds_detail)
                        st.dataframe(df_explore_specific[df_explore_specific['Seed'] == selected_seed_detail])
                    else: st.info(f"No runs with seeds for {explore_method_detail}.")
                else: st.dataframe(df_explore_specific)
            else: st.info(f"No data for method: {explore_method_detail}")
    with st.expander("Show/Hide Full Raw Data Table"): st.dataframe(results_df)
    st.sidebar.markdown("---");
else:
    st.error("🚨 Data could not be loaded. Please check the CSV file and its path (`CSV_FILE_PATH`).")

Overwriting dashboard.py


In [23]:
# Cell 4: Run Streamlit App and Expose with ngrok (Foreground Debugging Mode with ngrok checks)

from pyngrok import ngrok, conf
import subprocess
import os
import time

# --- Kill existing ngrok/streamlit processes (best effort) ---
try:
    print("Attempting to kill existing ngrok processes...")
    # Using pyngrok's kill method which is more reliable if pyngrok started them
    ngrok.kill()
    time.sleep(2) # Give it a moment to release resources
    # A more forceful kill if ngrok.kill() doesn't catch everything
    subprocess.run(['killall', '-q', 'ngrok'], check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Previous ngrok processes (if any) should be stopped.")
except Exception as e:
    print(f"Note: Error during cleanup of ngrok (might be normal if none were running): {e}")

try:
    print("Attempting to kill existing Streamlit processes (best effort)...")
    # This is a common way Streamlit is started from shell
    # It's not foolproof for all ways Streamlit might be running
    subprocess.run("pkill -f 'streamlit run dashboard.py'", shell=True, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("Previous Streamlit processes (if any) should be stopped.")
except Exception as e:
    print(f"Note: Error during cleanup of Streamlit (might be normal if none were running): {e}")

# --- Configure ngrok (Optional: if you have an authtoken and didn't run Cell 2) ---
# If you have an ngrok authtoken and ran Cell 2, pyngrok should pick it up.
# Alternatively, you can explicitly set it here if Cell 2 was skipped:
# YOUR_NGROK_AUTHTOKEN = "YOUR_ACTUAL_NGROK_AUTHTOKEN_HERE"
# if YOUR_NGROK_AUTHTOKEN != "YOUR_ACTUAL_NGROK_AUTHTOKEN_HERE":
#     conf.get_default().auth_token = YOUR_NGROK_AUTHTOKEN
#     print("Ngrok authtoken configured via pyngrok.conf.")
# else:
#     print("Ngrok authtoken not explicitly set in this cell. Relying on prior !ngrok authtoken command or no auth.")


# --- Start ngrok tunnel ---
public_url = None
ngrok_tunnel = None # To keep track of the tunnel object for cleaner disconnect

try:
    print("Attempting to start ngrok tunnel for Streamlit on port 8501...")
    # pyngrok will download ngrok binary if not found
    ngrok_tunnel = ngrok.connect(addr=8501, proto="http", bind_tls=True) # Streamlit default port is 8501
    public_url = ngrok_tunnel.public_url
    print(f"Ngrok tunnel established successfully!")
    print(f"  Public URL (HTTPS): {public_url}")
    print(f"  Local address being tunneled: {ngrok_tunnel.config['addr']}")
except Exception as e:
    print(f"ERROR: Ngrok connection failed: {e}")
    print("Attempting to check for existing tunnels...")
    try:
        tunnels = ngrok.get_tunnels()
        if tunnels:
            print("Found existing ngrok tunnels:")
            for tunnel in tunnels:
                print(f"  - {tunnel.public_url} (proto: {tunnel.proto}) -> {tunnel.config['addr']}")
                # If a suitable tunnel already exists for port 8501, try to use it
                if tunnel.proto in ['http', 'https'] and tunnel.config['addr'] in ['http://localhost:8501', 'localhost:8501', '127.0.0.1:8501', ':8501']:
                    public_url = tunnel.public_url # Use the public URL of the existing tunnel
                    ngrok_tunnel = tunnel # Keep track of this existing tunnel
                    print(f"Re-using existing tunnel: {public_url}")
                    break # Stop after finding a suitable one
        else:
            print("No existing ngrok tunnels found.")
    except Exception as e_tunnels:
        print(f"ERROR: Could not query existing ngrok tunnels: {e_tunnels}")

# --- Run Streamlit in the FOREGROUND ---
if public_url:
    print(f"\nTo access your Streamlit app, please open this URL in your browser: {public_url}")
    print("\nStarting Streamlit application in the foreground...")
    print("Logs from Streamlit will appear below.")
    print("The Colab cell will keep running until you manually stop it (e.g., click the stop button next to the cell).")
    print("-" * 70)

    # Execute Streamlit. This will block until Streamlit is stopped.
    # If dashboard.py has an error, it should print here.
    try:
        subprocess.run(["streamlit", "run", "dashboard.py"], check=True)
    except FileNotFoundError:
        print("ERROR: 'streamlit' command not found. Make sure Streamlit is installed correctly in the Colab environment.")
    except subprocess.CalledProcessError as e:
        print(f"ERROR: Streamlit process exited with an error (return code {e.returncode}).")
        if e.stdout:
            print("Streamlit stdout:\n", e.stdout.decode())
        if e.stderr:
            print("Streamlit stderr:\n", e.stderr.decode())
    except KeyboardInterrupt:
        print("\nStreamlit run interrupted by user (KeyboardInterrupt).")
    finally:
        print("-" * 70)
        print("Streamlit process has finished or been stopped.")
else:
    print("\nStreamlit application will NOT be started because an ngrok tunnel could not be established.")
    print("Please check ngrok error messages above.")

# --- Disconnect ngrok when done (after Streamlit stops) ---
if ngrok_tunnel: # If we successfully created or found a tunnel
    print("\nAttempting to disconnect ngrok tunnel...")
    try:
        ngrok.disconnect(ngrok_tunnel.public_url) # Disconnect specific tunnel
        print(f"Ngrok tunnel {ngrok_tunnel.public_url} disconnected.")
    except Exception as e:
        print(f"Warning: Error disconnecting ngrok tunnel {ngrok_tunnel.public_url if ngrok_tunnel else 'N/A'}: {e}")
        print("You might need to manually stop ngrok processes if issues persist or use ngrok.kill().")

# Final attempt to kill all ngrok processes to ensure cleanup
try:
    ngrok.kill()
    print("All ngrok processes have been requested to stop.")
except Exception as e:
    print(f"Warning: Error during final ngrok.kill(): {e}")

print("\nCell execution finished.")

Attempting to kill existing ngrok processes...
Previous ngrok processes (if any) should be stopped.
Attempting to kill existing Streamlit processes (best effort)...
Previous Streamlit processes (if any) should be stopped.
Attempting to start ngrok tunnel for Streamlit on port 8501...
Ngrok tunnel established successfully!
  Public URL (HTTPS): https://3225-34-48-157-79.ngrok-free.app
  Local address being tunneled: http://localhost:8501

To access your Streamlit app, please open this URL in your browser: https://3225-34-48-157-79.ngrok-free.app

Starting Streamlit application in the foreground...
Logs from Streamlit will appear below.
The Colab cell will keep running until you manually stop it (e.g., click the stop button next to the cell).
----------------------------------------------------------------------

Streamlit run interrupted by user (KeyboardInterrupt).
----------------------------------------------------------------------
Streamlit process has finished or been stopped.

At